In [1]:
import pymc3 as pm
import arviz as az
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

WARNING (theano.link.c.cmodule): install mkl with `conda install mkl-service`: No module named 'mkl'


# Gibbs Sampler Example 2: Pumps

Adapted from [Codes for Unit 5: pumpsmc.m and pumpsbugs.odc](https://www2.isye.gatech.edu/isye6420/supporting.html).

Associated lecture video: [Unit 5 Lesson 13](https://www.youtube.com/watch?v=DKlP93sD0O0&list=PLv0FeK5oXK4l-RdT6DWJj0_upJOG2WKNO&index=49).

Based on [Conjugate Likelihood Distributions, George, Makov, and Smith, 1993](https://www.jstor.org/stable/4616270) and [Robust Empirical Bayes Analyses of Event Rates, Gaver and O'Muircheartaigh, 1987](https://www.jstor.org/stable/1269878).

In [2]:
rng = np.random.default_rng(1)

obs = 100000
burn = 1000

# data from Gaver and O'Muircheartaigh, 1987
X = np.array([5, 1, 5, 14, 3, 19, 1, 1, 4, 22])
t = np.array([94.32, 15.52, 62.88, 125.76, 5.24, 31.44, 1.048, 1.048, 2.096, 10.48])
n = len(X)

# params
c = .1
d = 1

# inits
theta = np.ones(n)
beta = 1

thetas = np.zeros((obs, 10))
betas = np.zeros(obs)

for i in tqdm(range(obs)):
    theta = rng.gamma(shape=X+1, scale=1/(beta+t), size=n)
    sum_theta = np.sum(theta)
    
    beta = rng.gamma(shape=n + c, scale= 1/(sum_theta + d))
    
    thetas[i] = theta
    betas[i] = beta

thetas = thetas[burn:]
betas = betas[burn:]

print(f'{np.mean(thetas, axis=0)=}')
print(f'{np.std(thetas)=}')
print(f'{np.mean(betas)=}')
print(f'{np.std(betas)=}')

  0%|          | 0/100000 [00:00<?, ?it/s]

np.mean(thetas, axis=0)=array([0.06283613, 0.11838764, 0.0934282 , 0.11795504, 0.61044286,
       0.61054294, 0.87096223, 0.87103813, 1.48272156, 1.94964544])
np.std(thetas)=0.7294065045864947
np.mean(betas)=1.3367177163632642
np.std(betas)=0.4843747815427844


In [3]:
np.mean(thetas, axis=0)

array([0.06283613, 0.11838764, 0.0934282 , 0.11795504, 0.61044286,
       0.61054294, 0.87096223, 0.87103813, 1.48272156, 1.94964544])

Now for the PyMC model:

In [4]:
with pm.Model() as pumps:
    times = pm.Data('times', t)
    data = pm.Data('data', X)

    beta = pm.Gamma('beta', alpha=c, beta=d)
    theta = pm.Gamma('theta', alpha=d, beta=beta, shape=n)
    rate = pm.Deterministic('rate', theta * times)
    
    likelihood = pm.Poisson('likelihood', mu=rate, observed=data)
    
    # start sampling
    trace = pm.sample(5000,
                      chains=4,
                      tune=500,
                      init='jitter+adapt_diag',
                      random_seed=1,
                      cores=4,
                      return_inferencedata=True
                     )

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [theta, beta]
WARNING (theano.link.c.cmodule): install mkl with `conda install mkl-service`: No module named 'mkl'
WARNING (theano.link.c.cmodule): install mkl with `conda install mkl-service`: No module named 'mkl'
WARNING (theano.link.c.cmodule): install mkl with `conda install mkl-service`: No module named 'mkl'
WARNING (theano.link.c.cmodule): install mkl with `conda install mkl-service`: No module named 'mkl'


/Users/aaron/mambaforge/envs/pymc3_test/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc3_test/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc3_test/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc3_test/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
Sampling 4 chains for 500 tune and 5_000 draw iterations (2_000 + 20_000 draws total) took 11 seconds.


In [5]:
az.summary(trace, var_names=["~rate"])

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta,1.334,0.481,0.490,2.208,0.003,0.002,25961.0,16680.0,1.0
theta[0],0.063,0.026,0.018,0.109,0.000,0.000,31695.0,14454.0,1.0
theta[1],0.119,0.085,0.006,0.276,0.001,0.000,25426.0,12143.0,1.0
theta[2],0.094,0.038,0.032,0.167,0.000,0.000,30048.0,13379.0,1.0
theta[3],0.118,0.030,0.064,0.176,0.000,0.000,29740.0,14353.0,1.0
theta[4],0.611,0.305,0.123,1.173,0.002,0.001,29190.0,13566.0,1.0
theta[5],0.610,0.136,0.365,0.867,0.001,0.001,31222.0,14388.0,1.0
theta[6],0.866,0.646,0.021,2.023,0.004,0.003,23802.0,12778.0,1.0
theta[7],0.868,0.642,0.026,2.022,0.004,0.003,24846.0,12712.0,1.0
theta[8],1.483,0.694,0.355,2.759,0.004,0.003,25724.0,13904.0,1.0


Compare to BUGS results:

|           | mean    | sd      | MC_error | val2.5pc | median  | val97.5pc | start | sample |
|-----------|---------|---------|----------|----------|---------|-----------|-------|--------|
| beta      | 1.34    | 0.486   | 0.002973 | 0.5896   | 1.271   | 2.466     | 1001  | 50000  |
| theta[1]  | 0.06261 | 0.02552 | 1.11E-04 | 0.02334  | 0.05914 | 0.1217    | 1001  | 50000  |
| theta[2]  | 0.118   | 0.08349 | 3.69E-04 | 0.01431  | 0.09888 | 0.3296    | 1001  | 50000  |
| theta[3]  | 0.09366 | 0.03829 | 1.71E-04 | 0.03439  | 0.08842 | 0.1828    | 1001  | 50000  |
| theta[4]  | 0.1178  | 0.03048 | 1.47E-04 | 0.06595  | 0.115   | 0.1848    | 1001  | 50000  |
| theta[5]  | 0.6116  | 0.3097  | 0.001409 | 0.1632   | 0.5589  | 1.361     | 1001  | 50000  |
| theta[6]  | 0.6104  | 0.1366  | 6.45E-04 | 0.3705   | 0.6001  | 0.9058    | 1001  | 50000  |
| theta[7]  | 0.8686  | 0.6494  | 0.003059 | 0.101    | 0.7124  | 2.537     | 1001  | 50000  |
| theta[8]  | 0.8692  | 0.6481  | 0.003354 | 0.09784  | 0.7117  | 2.513     | 1001  | 50000  |
| theta[9]  | 1.478   | 0.6897  | 0.00351  | 0.4705   | 1.367   | 3.128     | 1001  | 50000  |
| theta[10] | 1.944   | 0.4133  | 0.002022 | 1.223    | 1.916   | 2.83      | 1001  | 50000  |